# WRF SHEAR VORTICITY perturbation inversion calculation

In [1]:
#Standard imports
import numpy as np
from numpy import *
import matplotlib.pyplot as plt
import netCDF4
from netCDF4 import Dataset
from wrf import tk, to_np, getvar, CoordPair, destagger, smooth2d, interplevel, vinterp, ll_to_xy, latlon_coords,get_cartopy,ALL_TIMES
import time
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import metpy.calc as mpcalc
from metpy.units import units

from wrf import (cartopy_xlim, cartopy_ylim, extract_times, interpz3d)

In [2]:
#Define type of tile and desired pressure level
#fullphys_expdom or nolatentheat_expdom
filetype = "fullphys_expdom"                                   #CHANGE

In [3]:
wrfout_fname = 'wrfout_'+filetype+'.nc'  #Name of WRF output file
wrfout = Dataset(wrfout_fname, 'r')  #Dataset is the class behavior to open the file

In [4]:
msfm = wrfout.variables['MAPFAC_M'][0, :, :]     #Map scale factor on mass grid
cor = wrfout.variables['F'][0,:, :]     #Coriolis parameter
ds = 1./wrfout.variables['RDX'][0]     #Grid spacing SAME IN ZONAL AND MERIDIONAL DIRECTIONS
slp = getvar(wrfout, "slp", timeidx=ALL_TIMES) #Get the sea level pressure
thta = getvar(wrfout, "th", timeidx=ALL_TIMES) #Get the potential temperature
z = getvar(wrfout, "z", units="m", timeidx=ALL_TIMES) #Get the heights in meters
T = getvar(wrfout,"tk", timeidx=ALL_TIMES) #Get the temperatures in K
lats, lons = latlon_coords(slp) #Get the latitude and longitude points
cart_proj = get_cartopy(slp) # Get the cartopy mapping object

In [5]:
#Define constants
R = 287.05 #J kg^-1 K^-1
kappa=0.2856219 #Rd / cp
cp = 1004.7 #J kg^-1 K^-1
pref = 1000. #hPa
g = 9.80665 #gravity
omega = 7.292e-5 #Earth's rotation rate (s^-1)
a = 2.e7 / np.pi #Radius of the earth (m)
p0=1.e5 #Reference pressure (100000 Pa)
alpha=-1./5.255877
gamma = 0.0065 #lapse rate, (K/m)
f0 = np.average(cor) #Coriolis force
dp = 50.e2 #Spacing of pressure levels (5000 Pa)

In [6]:
#Import python file to calculate standard atmospheric values
from standard_atmosphere import standard_atmosphere
pb = 1000. #Pressure at bottom level
pb_half = 975. #Half pressure at bottom level (pb + 0.5 * dp)
pt = 100. #Pressure at top level
pt_half = 125. #Half pressure at top level (pt + 0.5 * dp)
p_levels = np.linspace(pb,pt,19) #Define array of (19) pressure values starting from pb going to pt
p_half_levels = np.linspace(pb_half,pt_half,18) #Define array of half pressure levels starting from pb_half going to pt_half

#Call standard_atmosphere function and calculate
#ZS = standard heights (m)
#phiS = standard geopotential heights (m^2 / s^2)
#TS = standard temperature (K)
#SS = standard stability
ZS,phiS,TS,SS=standard_atmosphere(p_levels, p_half_levels) 

In [7]:
#Get times from dataset
times = extract_times(wrfout, None)
#Count times (used in plotting sequence)
count = np.count_nonzero(times)
#Set t, k, j, i indice lengths
tlen = len(times)
klen = len(p_levels)
jlen = np.shape(msfm)[0]
ilen = np.shape(msfm)[1]

In [8]:
#Format times to be in standard format (yyyy-mm-dd hh:ss)
dattimes = []
import pandas as pd
for i in range (0, count):
    dattimes.append(str(pd.Timestamp(times[i])))

In [ ]:
z_full = np.zeros((tlen,klen,jlen,ilen)) #Array of full heights (including those extrapolated below ground)
T_full = np.zeros((tlen,klen,jlen,ilen)) #Array of full temperature (including those extrapolated below ground)

#Extrapolate heights (m) and temperatures (K) below ground using wrf vinterp function
for t in np.arange(tlen):
    z_full[t,:,:,:] = vinterp(wrfout, z[t,:,:,:], 'pressure', p_levels, extrapolate=True, field_type='z', log_p=True, timeidx=t,meta=False)
    T_full[t,:,:,:] = vinterp(wrfout, T[t,:,:,:], 'pressure', p_levels, extrapolate=True, field_type='tk', log_p=True, timeidx=t,meta=False)

In [ ]:
#Function to calculate the geostrophic relative vorticity
def geostrophic_vorticity(tlen,num_levs,num_lats,num_lons,msfm,ds,f0,phi_prime):
    import numpy as np

    g=9.8066
    qgpv_geovor = np.zeros([tlen,num_levs,num_lats,num_lons])

    AC_1 = np.zeros([num_lats,num_lons])
    AC_2 = np.zeros([num_lats,num_lons])
    AC_3v = np.zeros([num_lats,num_lons])
    AC_4 = np.zeros([num_lats,num_lons])
    AC_5 = np.zeros([num_lats,num_lons])

    for j in np.arange(1,num_lats-1):
        for i in np.arange(1,num_lons-1): 
            AC_1[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_2[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_4[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_5[j,i] =    (msfm[j,i]**2.)/(f0*ds*ds)
            AC_3v[j,i] = (-4*msfm[j,i]**2.)/(f0*ds*ds)

    for t in np.arange(tlen):
        for j in np.arange(1,num_lats-1):
            for i in np.arange(1,num_lons-1):
                qgpv_geovor[t,:,j,i]=AC_1[j,i]*phi_prime[t,:,j-1,i]+AC_2[j,i]*phi_prime[t,:,j,i-1]+ \
                AC_3v[j,i]*phi_prime[t,:,j,i]+AC_4[j,i]*phi_prime[t,:,j,i+1]+AC_5[j,i]*phi_prime[t,:,j+1,i]

    return qgpv_geovor

In [ ]:
num_lats=lats.shape[0] #Number of latitude points (Same as jlen)
num_lons=lons.shape[1] #Number of longitude points (Same as ilen)
num_levs=p_levels.size #Number of vertical levels (Same as klen)

phi_prime=np.zeros([tlen,p_levels.size,num_lats,num_lons]) #Array of perturbation geopotential heights

#Calculate geopotential height perturbations by subtracting gravity times the full heights (z_full) and
#the standard atmospheric heights (ZS)
for t in np.arange(tlen):
    for k in np.arange(1,p_levels.size):
        phi_prime[t,k,:]=g*(z_full[t,k,:]-ZS[k])

#Use geostrophic_vorticity function to calculate geostrophic relative vorticity (qgpv_geovor)
qgpv_geovor=geostrophic_vorticity(tlen,num_levs,num_lats,num_lons,msfm,ds,f0,phi_prime)

In [ ]:
#Function to calculate the stretching term
def stretching_term(num_levs,num_lats,num_lons,msfm,ds,dp,f0,S,phip):

    AC_3s=np.zeros([num_levs])
    AC_6=np.zeros([num_levs])
    AC_7=np.zeros([num_levs]) 
    
    for k in np.arange(1,18):
        AC_6[k]=f0/(S[k-1]*dp**2.)
        AC_7[k]=f0/(S[k]  *dp**2.)
        AC_3s[k]= -(AC_6[k]+AC_7[k])
    strat=np.zeros([tlen,num_levs,num_lats,num_lons])
    
    for k in np.arange(1,num_levs-1):
        for j in np.arange(1,num_lats-1):
            for i in np.arange(1,num_lons-1):
                strat[t,k,j,i] = AC_3s[k]*phip[t,k,j,i]+ AC_6[k]*phip[t,k-1,j,i]+ AC_7[k]*phip[t,k+1,j,i]
    return strat

In [ ]:
#Use stretching_term function to calculate stretching vorticity (qgpv_strtch)
qgpv_strtch = stretching_term(num_levs,num_lats,num_lons,msfm,ds,dp,f0,SS,phi_prime)
#Add geostrophic relative vorticity and stretching vorticity term to get value of QGPV (qgpv)
qgpv = qgpv_geovor + qgpv_strtch

## Shear and curvature calculations

In [ ]:
#Get dx and dy from longitude and latitude arrays
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)
dx = np.asarray(dx)
dy = np.asarray(dy)
#Convert longitude and latitude arrays into numpy arrays
lats = np.asarray(lats)
lons = np.asarray(lons)

In [ ]:
ug = np.zeros((tlen,klen,jlen,ilen))
vg = np.zeros((tlen,klen,jlen,ilen))

for t in np.arange(tlen):
    for k in np.arange(klen):
        ug[t,k,:,:], vg[t,k,:,:] = mpcalc.geostrophic_wind(z_full[t,k,:,:]*units.meter, f0, dx*units.meter, dy*units.meter)

In [ ]:
ug_sq = ug ** 2. #Squared ug
vg_sq = vg ** 2. #Squared vg
vtot = np.sqrt((ug_sq + vg_sq)) #Total geostrophic wind

In [ ]:
shear = np.zeros((tlen, klen, jlen, ilen)) #Array for values of shear vorticity
curv = np.zeros((tlen, klen, jlen, ilen)) #Array for values of curvature vorticity

#Calculation of shear and curvature vorticity using centered-finite differencing as in Bell and Keyser 1993
for t in np.arange(tlen):
    print("Working on time: ", dattimes[t])
    for k in np.arange(klen):
        for j in np.arange(1, jlen-1):
            for i in np.arange(1, ilen-1):
                shear[t,k,j,i] = (-1. / (vtot[t,k,j,i] ** 2.)) * ((ug_sq[t,k,j,i] * (msfm[j,i] * (ug[t,k,j+1,i] - ug[t,k,j-1,i])/(2. * ds))) - (vg_sq[t,k,j,i] * (msfm[j,i] * (vg[t,k,j,i+1] - vg[t,k,j,i-1])/(2. * ds))) - (ug[t,k,j,i] * vg[t,k,j,i] * ((msfm[j,i] * (ug[t,k,j,i+1] - ug[t,k,j,i-1])/(2. * ds)) - (msfm[j,i] * (vg[t,k,j+1,i] - vg[t,k,j-1,i])/(2. * ds)))))
                curv[t,k,j,i] = (1. / (vtot[t,k,j,i] ** 2.)) * ((ug_sq[t,k,j,i] * (msfm[j,i] * (vg[t,k,j,i+1] - vg[t,k,j,i-1])/(2. * ds))) - (vg_sq[t,k,j,i] * (msfm[j,i] * (ug[t,k,j+1,i] - ug[t,k,j-1,i])/(2. * ds))) - (ug[t,k,j,i] * vg[t,k,j,i] * ((msfm[j,i] * (ug[t,k,j,i+1] - ug[t,k,j,i-1])/(2. * ds)) - (msfm[j,i] * (vg[t,k,j+1,i] - vg[t,k,j-1,i])/(2. * ds)))))
        

In [ ]:
#Calculation of relative vorticity
rel_vor = np.zeros((tlen, klen, jlen, ilen))

for t in np.arange(tlen):
    print("Working on time: ", dattimes[t])
    for k in np.arange(klen):
        rel_vor[t,k,:,:] = mpcalc.vorticity(ug[t,k,:,:],vg[t,k,:,:],dx*units.meter, dy*units.meter,dim_order='yx')
        

In [ ]:
#Add shear and curvature vorticity, should equal qgpv_geovor
shrpluscurv = shear + curv

In [ ]:
shear_mean = np.zeros((klen,jlen,ilen)) #Array of mean values of shear vorticity

#Calculate mean of shear vorticity using np.mean and np.squeeze
for k in np.arange(klen):
    for j in np.arange(jlen):
        for i in np.arange(ilen):
            shear_mean[k,j,i]= np.squeeze(np.mean(shear[0:,k,j,i])) #Taking time mean of QGPV

In [ ]:
shear_p = np.zeros([tlen,num_levs,num_lats,num_lons]) #Array of perturbation values of shear vorticity

#Calculate shear vorticity perturbations by subtracting shear vorticity from mean shear vorticity
for t in np.arange(tlen):
    for k in np.arange(klen):
        shear_p[t,k,:] = shear[t,k,:]-shear_mean[k,:]

In [ ]:
zper = np.zeros((tlen,klen,jlen,ilen)) #Array for height perturbations

#Calculate height perturbations by subtracting the extrapolated heights (z_full) and the standard atmosphere heights (ZS)
for t in np.arange(tlen):
    for k in np.arange(klen):
        zper[t,k,:,:] = (z_full[t,k,:,:] - ZS[k])

In [ ]:
zper_mean = np.zeros((klen,jlen,ilen)) #Array for height perturbation mean

#Calculate mean height perturbations using np.mean and np.squeeze
for k in np.arange(klen):
    for j in np.arange(jlen):
        for i in np.arange(ilen):
            zper_mean[k,j,i]= np.squeeze(np.mean(zper[0:,k,j,i]))

In [ ]:
%%time
z_prime = np.zeros((tlen,klen,jlen,ilen)) #Array for phi'' of 

#Calculate ____ (z_prime) by subtracting the height perturbation (zper) from the heigh perturbation mean (zper_mean)
for t in np.arange(tlen):
    for k in np.arange(klen):
        z_prime[t,k,:,:] = zper[t,k,:,:] - zper_mean[k,:,:]

In [ ]:
#Calculate the thickness of the bottom level (dz_b) and the top level (dz_t)
dz_b = (z_prime[:,1,:,:]-z_prime[:,0,:,:])
dz_t = (z_prime[:,-1,:,:]-z_prime[:,-2,:,:])

#Calculate the perturbation temperature at the bottom level (Tb) and the top level (Tt)
Tb = ((g*dz_b)/R)/np.log(p_levels[0]/p_levels[1])
Tt = ((g*dz_t)/R)/np.log(p_levels[-2]/p_levels[-1])

In [ ]:
#Function to invert the QGPV vorticity
def qgpv_invert(iterations,tlen,num_levs,num_lats,num_lons,p_levels,f0, ds, dp,msfm, SS,qgpv, Tb, Tt, BC, BC_flag):   
    
    import numpy as np
    
    R = 287.04
    g = 9.8066
    
#BC_flag = 0 for Dirichlet and 1 for Neumann
    res=np.zeros([tlen,num_levs,num_lats,num_lons])
    phip = BC
    

# define coefficients

    AC_3s=np.zeros([num_levs])
    AC_6=np.zeros([num_levs])
    AC_7=np.zeros([num_levs]) 
    
    for k in np.arange(17):
        AC_6[k+1]=f0/(SS[k]*dp**2.)
        AC_7[k+1]=f0/(SS[k+1]  *dp**2.)
        AC_3s[k+1]= -(AC_6[k+1]+AC_7[k+1])

    mfs=np.zeros([num_lats,num_lons])
    mfs[:] = (f0*ds*ds)/(msfm[:]**2.)

    AI_1 = 1.
    AI_2 = 1.
    AI_4 = 1.
    AI_5 = 1.
    AI_3v = -4.

    AI_3s=np.zeros([num_levs,num_lats,num_lons])
    AI_6=np.zeros([num_levs,num_lats,num_lons])
    AI_7=np.zeros([num_levs,num_lats,num_lons])

    for k in np.arange(num_levs):
        AI_6[k,:]  = mfs[:]*AC_6[k]
        AI_7[k,:]  = mfs[:]*AC_7[k] 
        AI_3s[k,:] = mfs[:]*AC_3s[k]

#####
    for t in np.arange(tlen):
        for num_iter in np.arange(iterations):
            for k in np.arange(1,num_levs-1):
                for j in np.arange(1,num_lats-1):
                    for i in np.arange(1,num_lons-1):
                        RES = AI_1*phip[t,k,j-1,i] + \
                        AI_2*phip[t,k,j,i-1] + \
                        (AI_3v+AI_3s[k,j,i])*phip[t,k,j,i] + \
                        AI_4*phip[t,k,j,i+1] + \
                        AI_5*phip[t,k,j+1,i] + \
                        AI_6[k,j,i]*phip[t,k-1,j,i] + \
                        AI_7[k,j,i]*phip[t,k+1,j,i] - qgpv[t,k,j,i]*mfs[j,i]
                
                        phip[t,k,j,i] = phip[t,k,j,i] - 1.8*RES/(AI_3v+AI_3s[k,j,i])
                        res[t,k,j,i]=RES
                        
                if(np.amax(res/g)<.25):  
                    print('stopping at iteration number: ', num_iter)
                    break
                        
            if(BC_flag==1):
                if(k==1): 
                    phip[t,0,:] = phip[t,1,:] - (R * Tb[t,:]/(100*p_levels[0]))*dp
                if(k==num_levs-2): 
                    phip[t,-1,:] = phip[t,-2,:] + (R * Tt[t,:]/(100.*p_levels[-1]))*dp

    
    return phip,res,num_iter

In [ ]:
%%time
iterations=50
#BC effectivley sets first guess for inversion (here it is set to zero everywhere)
BC =np.zeros([tlen,p_levels.size,num_lats,num_lons])
res=np.zeros([tlen,p_levels.size,num_lats,num_lons])

#qgpv_anomaly = np.zeros([p_levels.size,num_lats,num_lons])
#for k in range(p_levels.size):
#    BC[k,:]=9.81*(z_full[k,:]-ZS[k])
BC_flag=1
Tb=np.zeros([tlen,num_lats,num_lons])
Tt=np.zeros([tlen,num_lats,num_lons])


phip=np.zeros([p_levels.size,num_lats,num_lons])
phip,res,num_iter=qgpv_invert(iterations,tlen,num_levs,num_lats,num_lons,p_levels,f0, ds, dp,msfm,SS,shear_p, Tb, Tt, BC, BC_flag)

In [ ]:
print("Number of iterations: ", num_iter+1)
print("Average residual: ", np.mean(res))  
print("Absolute maximum residual: ", np.amax(res))

In [ ]:
#Save data to file for use in other notebooks
import pickle
with open('inv_shear_full_'+filetype+'.pkl', 'wb') as f:
    pickle.dump([phip,shear,curv], f)

In [ ]:
#Load data from saved file
#import pickle
#with open('inv_shear_full_'+filetype+'.pkl', 'rb') as f:
    #phip,shear,curv = pickle.load(f)
    
#phip = np.squeeze(phip)

In [ ]:
#Get dx and dy from longitude and latitude arrays
dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)
#Convert longitude and latitude arrays into numpy arrays
lats = np.asarray(lats)
lons = np.asarray(lons)

In [ ]:
#Rename longitude and latitude arrays for correcting cartopy plotting issue
lats_redo = lats
lons_redo = lons

In [ ]:
#Account for cartopy plotting issue when longitude is less than 0.0
for j in range(jlen):
    for i in range(ilen):
        if lons_redo[j,i] < 0.0: #If longitude is negative, add 360 degrees to it to get positive longitude
            lons_redo[j,i] = lons_redo[j,i] + 360.
        else:
            lons_redo[j,i] = lons_redo[j,i]

In [ ]:
#Set revised longitude and latitude arrays as true longitude and latitude arrays
lats = lats_redo
lons = lons_redo

In [ ]:
#Input vertical level (hPa) in which you want to plot
levtitle = str(input('Enter desired level for plotting (300, 500, 850, or 950): '))

In [ ]:
#Choose level of plotting based on levtitle
if levtitle == "300":
    lev = 14
elif levtitle == "500":
    lev = 10
elif levtitle == "850":
    lev = 3
elif levtitle == "900":
    lev = 2
elif levtitle == "950":
    lev = 1
elif levtitle == "1000":
    lev = 0

In [ ]:
#calculate winds determined by qgpv perturbation from inversion
#U = np.zeros([tlen,klen,num_lats,num_lons])
#V = np.zeros([tlen,klen,num_lats,num_lons])

#for t in np.arange(tlen):
    #for k in np.arange(klen):
        #for j in np.arange(1,num_lats-1):
            #for i in np.arange(1,num_lons-1):
                #U[t,k,j,i] = -msfm[j,i]*0.5*(phip[t,k,j+1,i]-phip[t,k,j-1,i])/(f0*ds)
                #V[t,k,j,i] =  msfm[j,i]*0.5*(phip[t,k,j,i+1]-phip[t,k,j,i-1])/(f0*ds)

In [ ]:
#Import smoothing function
from scipy.ndimage.filters import gaussian_filter

In [ ]:
def plot_maxmin_points(lon, lat, data, data2, extrema, nsize, symbol, color='k',
                     plotValue=True, transform=None):
    """
    This function will find and plot relative maximum and minimum for a 2D grid. The function
    can be used to plot an H for maximum values (e.g., High pressure) and an L for minimum
    values (e.g., low pressue). It is best to used filetered data to obtain  a synoptic scale
    max/min value. The symbol text can be set to a string value and optionally the color of the
    symbol and any plotted value can be set with the parameter color
    lon = plotting longitude values (2D)
    lat = plotting latitude values (2D)
    data = 2D data that you wish to plot the max/min symbol placement
    data2 = 2D data that you wish to plot the value of at the max/min symbol
    extrema = Either a value of max for Maximum Values or min for Minimum Values
    nsize = Size of the grid box to filter the max and min values to plot a reasonable number
    symbol = String to be placed at location of max/min value
    color = String matplotlib colorname to plot the symbol (and numerica value, if plotted)
    plot_value = Boolean (True/False) of whether to plot the numeric value of max/min point
    The max/min symbol will be plotted on the current axes within the bounding frame
    (e.g., clip_on=True)
    """
    from scipy.ndimage.filters import maximum_filter, minimum_filter

    if (extrema == 'max'):
        data_ext = maximum_filter(data, nsize, mode='nearest')
    elif (extrema == 'min'):
        data_ext = minimum_filter(data, nsize, mode='nearest')
    else:
        raise ValueError('Value for hilo must be either max or min')

    mxy, mxx = np.where(data_ext == data)

    for i in range(len(mxy)):
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i], mxx[i]], symbol, color=color, size=28,
                clip_on=True, horizontalalignment='center', verticalalignment='center',
                transform=transform,zorder=10)
        #Prints value of data2 at extrema
        props = dict(facecolor = 'white',pad=1)
        ax.text(lon[mxy[i], mxx[i]], lat[mxy[i]-3, mxx[i]],
                str(np.int(data2[mxy[i], mxx[i]])),
                color=color, size=12, bbox=props, clip_on=True, fontweight='bold',
                horizontalalignment='center', verticalalignment='top', transform=transform,zorder=10)

In [ ]:
#PLOTTING FOR GEOPOTENTIAL HEIGHT ANOMALY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = phip[i,lev,:,:]
    tmp2 = slp[i,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-400,-380,-360,-340,-320,-300,-280,-260,-240,-220,-200,-180,-160,-140,
              -120,-100,-80,-60,-40,-20,20,40,60,80,100,120,140,
              160,180,200,220,240,260,280,300]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="gray")
    plt.clabel(qgcts, fmt='%.1f', inline=True,inline_spacing=-2.)

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3),gaussian_filter(to_np(tmp1),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())
    
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Inverted Geopotential Height Anomaly (m^2 / s^2) from full Shear Vorticity Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/shear/geopanom/geopanom_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR HEIGHT ANOMALY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = phip[i,lev,:,:] / g
    tmp2 = slp[i,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-60,-56,-52,-48,-44,-40,-36,-32,-28,-24,-20,-16,-12,-8,-4,
              4,8,12,16,20,24,28,32,36,40,44,48,52,56,60]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="gray")
    plt.clabel(qgcts, fmt='%.1f', inline=True, inline_spacing=-2.)

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3),gaussian_filter(to_np(tmp1),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())

    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Inverted Height Anomaly (m) from full Shear Vorticity Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/shear/hghtanom/hghtanom_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR SHEAR VORTICITY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = shear[i,lev,:,:] * 10.e4
    tmp2 = slp[i,:,:]
    tmp3 = z_full[i,lev,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-60,-56,-52,-48,-44,-40,-36,-32,-28,-24,-20,-16,-12,-8,-4,
              4,8,12,16,20,24,28,32,36,40,44,48,52,56,60]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="gray")
    plt.clabel(qgcts, fmt='%.1f', inline=True, inline_spacing=-2.)

    #Option to plot heights
    if lev == 10:
        levels1 = np.arange(4000,7000,150)
    if lev == 1:
        levels1 = np.arange(100,900,50)
    hghtcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp3),3),levels=levels1, transform=crs.PlateCarree(), colors="blue")
    plt.clabel(hghtcts, fmt='%.1f', inline=True, inline_spacing=-2.)
    
    #Option to plot geostrophic winds
    #Q = ax.barbs(to_np(lons_redo[::10,::10]),to_np(lats_redo[::10,::10]), \
        #ug[i,lev,::10,::10], vg[i,lev,::10,::10],\
        #transform=crs.PlateCarree())

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3),gaussian_filter(to_np(tmp1),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Shear Vorticity (1/s, Scaled 10^4) from full Shear Vorticity Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/shear/shearvor/shearvor_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR SHEAR overlaid with CURVATURE VORTICITY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = shear[i,lev,:,:] * 10.e4
    tmp2 = slp[i,:,:]
    tmp3 = curv[i,lev,:,:] * 10.e4
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-60,-56,-52,-48,-44,-40,-36,-32,-28,-24,-20,-16,-12,-8,-4,
              4,8,12,16,20,24,28,32,36,40,44,48,52,56,60]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="gray")
    plt.clabel(qgcts, fmt='%.1f', inline=True, inline_spacing=-2.)
    
    qgccts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp3),3), levels=levels, transform=crs.PlateCarree(), colors="red")
    plt.clabel(qgccts, fmt='%.1f', inline=True, inline_spacing=-2.)

    
    #Option to plot geostrophic winds
    #Q = ax.barbs(to_np(lons_redo[::10,::10]),to_np(lats_redo[::10,::10]), \
        #ug[i,lev,::10,::10], vg[i,lev,::10,::10],\
        #transform=crs.PlateCarree())

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3),gaussian_filter(to_np(tmp1),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Shear Vorticity (gray) and Curvature Vorticity (red) (1/s, Scaled 10^4) from full Vorticity Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/shearpluscurv/shear_over_curv_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()

In [ ]:
#PLOTTING FOR SHEAR and CURVATURE VORTICITY
for i in range (tlen):
    print("Working on time: ", dattimes[i])
    
    # Get the map projection information
    cart_proj = get_cartopy(wrfin=wrfout)
    
    # Create the figure
    fig = plt.figure(figsize=(12,9))
    ax = plt.axes(projection=cart_proj)
    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",
                             facecolor="none",
                             name="admin_1_states_provinces_shp")
    ax.add_feature(states, linewidth=0.5, edgecolor="black")
    ax.coastlines('50m', linewidth=0.8)

    tmp1 = shrpluscurv[i,lev,:,:] * 10.e4
    tmp2 = slp[i,:,:]
    
    #FOR EASY, COLORFUL VISUALIZATION
    #qgcs = plt.pcolormesh(to_np(lons_redo), to_np(lats_redo), to_np(tmp1),
                                #cmap='bwr', vmin=-100, vmax=100,
                                #transform=crs.PlateCarree())
    #plt.colorbar(qgcs, ax=ax, orientation="horizontal", pad=.05)
    
    #FOR PROPER, PAPER-READY PLOTS
    levels = [-60,-56,-52,-48,-44,-40,-36,-32,-28,-24,-20,-16,-12,-8,-4,
              4,8,12,16,20,24,28,32,36,40,44,48,52,56,60]
    qgcts = plt.contour(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp1),3), levels=levels, transform=crs.PlateCarree(), colors="purple")
    plt.clabel(qgcts, fmt='%.1f', inline=True, inline_spacing=-2.)

    
    #Option to plot geostrophic winds
    #Q = ax.barbs(to_np(lons_redo[::10,::10]),to_np(lats_redo[::10,::10]), \
        #ug[i,lev,::10,::10], vg[i,lev,::10,::10],\
        #transform=crs.PlateCarree())

    plot_maxmin_points(to_np(lons_redo), to_np(lats_redo), gaussian_filter(to_np(tmp2),3),gaussian_filter(to_np(tmp1),3), 'min', 25, symbol='L', color='black', transform=crs.PlateCarree())
    
    # Set the map bounds
    ax.set_xlim(cartopy_xlim(z))
    ax.set_ylim(cartopy_ylim(z))

    ax.gridlines(color="black", linestyle="dotted")

    #Zoom in on object storm
    ax.set_extent([-160, -110, 35, 55]) #lon,lon,lat,lat

    ax.set_title('WRF '+levtitle+'-hPa Shear + Curvature Vorticity (1/s, Scaled 10^4) from full Vorticity Perturbation \n Valid Time: {}'.format(dattimes[i]))
    #($\frac{m}{s^{-2}}$)

    plt.savefig('plots/inv_full/'+filetype+'/shearpluscurv/shear+curv_'+levtitle+'_'+filetype+'_{}.pdf'.format(dattimes[i]))
    plt.close()